In [1]:
import os
!pwd

/Users/tejasgadi/Local_Disk_D/Machine_Learning_Projects/finetuning_distilbert_text_classification/research


In [2]:
# Goto Root directory of the project
os.chdir("../")
!pwd

/Users/tejasgadi/Local_Disk_D/Machine_Learning_Projects/finetuning_distilbert_text_classification


In [4]:
# Model trauiner Config Schema/Data Class
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_path: Path

In [5]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories
from box import ConfigBox

## Config Manager

In [6]:
# Create Configuration Manager: This has basic information required before staring any module/components

class ConfigurationManager:
    def __init__(self, config_path = CONFIG_FILE_PATH, params_path = PARAMS_FILE_PATH):
        self.config:ConfigBox = read_yaml(config_path)
        self.params:ConfigBox = read_yaml(params_path).TrainingArguments

        # Create Artifacts Root dir
        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self)->ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_path =  config.metric_file_path,
        )

        return model_evaluation_config


In [8]:
## Model Evaluation libraries import
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_from_disk
import torch
import pandas as pd
from tqdm import tqdm
import evaluate

## Model Evaluation component

In [13]:
class ModelEvaluation:
    def __init__(self, data_transformation_config: ModelEvaluationConfig):
        self.config = data_transformation_config

    def generate_batch_sized_chunks(self,list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    def calculate_metric_on_test_ds(self,dataset, metric, model, tokenizer, device, batch_size=16,column_text="article", column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):

            inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                            padding="max_length", return_tensors="pt")

            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device),
                            length_penalty=0.8, num_beams=8, max_length=128)
            ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

            # Finally, we decode the generated texts,
            # replace the  token, and add the decoded texts with the references to the metric.
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                    clean_up_tokenization_spaces=True)
                for s in summaries]

            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


            metric.add_batch(predictions=decoded_summaries, references=target_batch)

        #  Finally compute and return the ROUGE scores.
        score = metric.compute()
        return score

    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)  #load a tokenizer

        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

        dataset_samsum = load_from_disk(self.config.data_path)
       
        rouge_metric = evaluate.load('rouge')
        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

        score = self.calculate_metric_on_test_ds(
            dataset_samsum['test'][0:10], rouge_metric, model_pegasus, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary', device=device
        )

        # Directly use the scores without accessing fmeasure or mid
        rouge_dict = {rn: score[rn] for rn in rouge_names}

        # Convert the dictionary to a DataFrame for easy visualization
        df = pd.DataFrame(rouge_dict, index=[f'pegasus'])
        df.to_csv(self.config.metric_file_path, index=False)

In [14]:
# Pipeline initiate

config = ConfigurationManager()
model_evaluation_config = config.get_model_evaluation_config()

model_evaluation = ModelEvaluation(model_evaluation_config)

model_evaluation.evaluate()

[2025-04-28 20:03:21,050 : INFO: common: yaml file: config/config.yaml is loaded sucessfully]
[2025-04-28 20:03:21,051 : INFO: common: yaml file: params.yaml is loaded sucessfully]
[2025-04-28 20:03:21,052 : INFO: common: Created directory at: artifacts]
[2025-04-28 20:03:21,052 : INFO: common: Created directory at: artifacts/model_evaluation]


100%|██████████| 5/5 [01:52<00:00, 22.43s/it]

[2025-04-28 20:05:18,891 : INFO: rouge_scorer: Using default tokenizer.]
